In [5]:
# Setting up the environment
import simpy
import random
import logging
import statistics
from dataclasses import dataclass
from datetime import timedelta
from itertools import groupby, product

logging.basicConfig(level=logging.DEBUG)

In [17]:
"""
Stating Interest stage of ATS onboarding flow.

Scenario:
  A potential interested customer lands on the ATS site and is interested in learning more. 
  The customer contacts ATS engineers (get_help_eng) to inquire details about ATS. 
  The customer uses the knowledge gained to go research the needs of their service and to complete prerequisites for onboarding
  The customer can elect to have a meeting with ATS engineers to run over their assumptions and metrics (on_call_eng).
  Alternatively, the customer can skip the meeting and submit a ticker for verification.

Goal the customer wants to wait a max of <TBD> hours

"""

class HelpDesk(object):
    def __init__(self, env, num_help_desk_eng, num_on_call_eng, num_verify_eng):
    # simulation environment
        self.env = env
        self.help_desk_eng = simpy.Resource(env, num_help_desk_eng)
        self.on_call_eng = simpy.Resource(env, num_on_call_eng)
        self.verify_eng = simpy.Resource(env, num_verify_eng)

    # resources and processes
    def get_help(self, customer):
        yield self.env.timeout(random.randint(1, 3))
    
    def collect_complete(self, customer):
        yield self.env.timeout(3 / 60)
    
    def verify_ticket(self, customer):
        yield self.env.timeout(random.randint(1, 5))

# moving through environment
def go_to_help_desk(env, customer, help_desk, wait_times):
    # used to keep track of how long it takes customer to move through system
    arrival_time = env.now

    # customer arriving at help-desk and waiting for guidance
    with help_desk.help_desk_eng.request() as request:
        yield request
        yield env.process(help_desk.get_help(customer))

    # customer runs through collecting and completing requirements
    with help_desk.on_call_eng.request() as request:
        yield request
        yield env.process(help_desk.collect_complete(customer))

    # randomness of meeting being requested or not.
    if random.choice([True, False]):
        with help_desk.on_call_eng.request() as request:
            yield request
            yield env.process(help_desk.collect_complete(customer))

    # customer submits ADO ticket
    wait_times.append(env.now - arrival_time)

# main process
def run_help_desk(env, num_help_desk_eng, num_on_call_eng, num_verify_eng, wait_times):
    help_desk = HelpDesk(env, num_help_desk_eng, num_on_call_eng, num_verify_eng)

    # customers waiting to be engaged as soon as help-desk opens
    for customer in range(3):
        env.process(go_to_help_desk(env, customer, help_desk, wait_times))

    # wait some time before generating a new customer
    for _ in range(27): # suppose a new customer arrives, on average, every 3 days
        yield env.timeout(0.20)
        customer +=1
        env.process(go_to_help_desk(env, customer, help_desk, wait_times))
    
    return(wait_times)

# calculate wait times
def calculate_wait_time(wait_times):
    #print([round(t) for t in wait_times])
    average_wait = statistics.mean(wait_times)
    # pretty print
    return(timedelta(minutes=average_wait))
    #days, frac_days = divmod(average_wait, 1)
    #hours = frac_days * 24
    #return (round(days), round(hours))

# main function
def run_simulation(num_help_desk_eng, num_on_call_eng, num_verify_eng):
    # setup
    wait_times = []
    
    # run simulation
    env = simpy.Environment()
    env.process(
        run_help_desk(env, num_help_desk_eng, num_on_call_eng, num_verify_eng, wait_times)
    )
    env.run()
    return(wait_times)

def calculate_average_wait_time_for_config(num_help_desk_eng, num_on_call_eng, num_verify_eng, num_simulations):
    all_wait_times = []
    for _ in range(num_simulations):
        all_wait_times += run_simulation(num_help_desk_eng, num_on_call_eng, num_verify_eng)

    return(calculate_wait_time(all_wait_times).total_seconds())

def generate_eng_config(max_engineers):
    num_places = 3
    return(
        config 
        for config in product(range(1, max_engineers + 1), repeat=num_places)
        if sum(config) <= max_engineers 
    )


def main():
    # Setup
    random.seed(2022)

    # Run simulation
    average_time_by_engineer_config = []

    max_engineers = 5
    num_simulations = 10

    for num_help_desk_eng, num_on_call_eng, num_verify_eng in generate_eng_config(max_engineers):
        average_time = calculate_average_wait_time_for_config(
            num_help_desk_eng, num_on_call_eng, num_verify_eng, num_simulations,
        )
        average_time_by_engineer_config.append(
            EngineerConfig(num_help_desk_eng, num_on_call_eng, num_verify_eng, average_time)
        )

    def total_engineers_key(e):
        return(e.total_engineers)

    average_time_by_engineer_config.sort(key=total_engineers_key)
    for k, g in groupby(average_time_by_engineer_config, total_engineers_key):
        if True:
            g = list(g)
            for group_item in g:
                logging.debug(group_item)
        best_config = min(g, key=lambda e: e.average_time)
        logging.info(f"For {k} engineers, best config is {best_config}\n")

@dataclass
class EngineerConfig:
    num_help_desk_eng: int 
    num_on_call_eng: int
    num_verify_eng: int

    average_time: float

    @property
    def total_engineers(self):
        return(sum((self.num_help_desk_eng, self.num_on_call_eng, self.num_verify_eng)))


if __name__ == '__main__':
    main()


DEBUG:root:EngineerConfig(num_help_desk_eng=1, num_on_call_eng=1, num_verify_eng=1, average_time=1764.77)
INFO:root:For 3 engineers, best config is EngineerConfig(num_help_desk_eng=1, num_on_call_eng=1, num_verify_eng=1, average_time=1764.77)

DEBUG:root:EngineerConfig(num_help_desk_eng=1, num_on_call_eng=1, num_verify_eng=2, average_time=1687.64)
DEBUG:root:EngineerConfig(num_help_desk_eng=1, num_on_call_eng=2, num_verify_eng=1, average_time=1751.3)
DEBUG:root:EngineerConfig(num_help_desk_eng=2, num_on_call_eng=1, num_verify_eng=1, average_time=804.5)
INFO:root:For 4 engineers, best config is EngineerConfig(num_help_desk_eng=2, num_on_call_eng=1, num_verify_eng=1, average_time=804.5)

DEBUG:root:EngineerConfig(num_help_desk_eng=1, num_on_call_eng=1, num_verify_eng=3, average_time=1692.1)
DEBUG:root:EngineerConfig(num_help_desk_eng=1, num_on_call_eng=2, num_verify_eng=2, average_time=1756.23)
DEBUG:root:EngineerConfig(num_help_desk_eng=1, num_on_call_eng=3, num_verify_eng=1, average_ti